In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('homework_1.1.csv')


In [4]:
df.head()

,Unnamed: 0,X1,X2,X3,Y
0,0,-0.440646,-0.390227,0.156718,-0.877671
1,1,-3.810099,-1.304665,-1.105117,-10.130388
2,2,-1.425451,-0.340049,1.115908,0.284068
3,3,-1.325750,0.161906,-0.254670,-1.994344
4,4,3.120263,1.487343,-1.164839,2.030030


In [7]:
import pandas as pd
import statsmodels.api as sm

# Assume df is your DataFrame with columns x1, x2, x3, and Y
X = df[['X1', 'X2', 'X3']]
X = sm.add_constant(X)  # adds the intercept term
y = df['Y']

# Fit the model
model = sm.OLS(y, X).fit()

# View results with t-statistics
print(model.summary())

# If you just want t-statistics:
print(model.tvalues)

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 3.543e+04
Date:                Mon, 26 May 2025   Prob (F-statistic):               0.00
Time:                        02:04:42   Log-Likelihood:                -727.62
No. Observations:                1000   AIC:                             1463.
Df Residuals:                     996   BIC:                             1483.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0026      0.016      0.166      0.8

In [8]:
import statsmodels.api as sm

simple_betas = {}
for xi in ['X1', 'X2', 'X3']:
    Xi = sm.add_constant(df[[xi]])
    yi = df['Y']
    model = sm.OLS(yi, Xi).fit()
    simple_betas[xi] = model.params[xi]

# Multiple regression
X_multi = sm.add_constant(df[['X1', 'X2', 'X3']])
y = df['Y']
multi_model = sm.OLS(y, X_multi).fit()
multi_betas = multi_model.params[['X1', 'X2', 'X3']].to_dict()

# Compute differences
diffs = {xi: abs(simple_betas[xi] - multi_betas[xi]) for xi in ['X1', 'X2', 'X3']}

# Find the Xi with the greatest difference
max_diff_xi = max(diffs, key=diffs.get)
print(f"{max_diff_xi} has the greatest difference: {diffs[max_diff_xi]:.4f}")


X2 has the greatest difference: 2.1190


In [11]:
multi_betas

{'X1': 1.0071376550530315, 'X2': 1.964568594874394, 'X3': 2.9754885351634353}

In [9]:
simple_betas

{'X1': np.float64(1.8417610991461832),
 'X2': np.float64(4.083612579373998),
 'X3': np.float64(3.0970412020498452)}

In [10]:
diffs

{'X1': np.float64(0.8346234440931517),
 'X2': np.float64(2.119043984499604),
 'X3': np.float64(0.12155266688640998)}

In [12]:
df = pd.read_csv('homework_1.2.csv')


In [19]:
df

,Unnamed: 0,X,Y,Z
0,0,0,0.548814,0.548814
1,1,1,1.215189,0.715189
2,2,0,0.602763,0.602763
3,3,0,0.544883,0.544883
4,4,0,0.423655,0.423655
...,...,...,...,...
95,95,0,0.183191,0.183191
96,96,1,1.086513,0.586513
97,97,0,0.020108,0.020108
98,98,1,1.328940,0.828940


In [14]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Assume df is your DataFrame with columns X, Y, Z
treated = df[df['X'] == 1].copy()
control = df[df['X'] == 0].copy()

# Fit NearestNeighbors on control group using Z
nn = NearestNeighbors(n_neighbors=1, algorithm='auto')
nn.fit(control[['Z']])

# Find nearest control unit for each treated unit
distances, indices = nn.kneighbors(treated[['Z']])

# Add matched control index to treated DataFrame
treated['matched_control_index'] = control.iloc[indices.flatten()].index.values
treated['matched_control_Z'] = control.iloc[indices.flatten()]['Z'].values

# Optionally: merge with control Y values if needed
treated['matched_control_Y'] = control.iloc[indices.flatten()]['Y'].values


In [15]:
distances, indices = nn.kneighbors(treated[['Z']])


In [16]:
farthest_distance = distances.max()
print(f"The distance of the farthest match is: {farthest_distance:.4f}")


The distance of the farthest match is: 0.2102


In [17]:
distances

array([[0.00113784],
       [0.00721421],
       [0.00055526],
       [0.11361625],
       [0.18550601],
       [0.03122042],
       [0.01356829],
       [0.14743989],
       [0.05446309],
       [0.0918554 ],
       [0.20046159],
       [0.02100181],
       [0.00483141],
       [0.00237243],
       [0.0131873 ],
       [0.16651217],
       [0.00392306],
       [0.00038939],
       [0.0007015 ],
       [0.00483827],
       [0.16559133],
       [0.01118243],
       [0.04093895],
       [0.01869601],
       [0.00287156],
       [0.00255483],
       [0.04514182],
       [0.00215221],
       [0.00156956],
       [0.21021709],
       [0.00233993],
       [0.00517034],
       [0.01229439],
       [0.04593677],
       [0.04283648],
       [0.05978816],
       [0.19830271],
       [0.19860434],
       [0.00208214],
       [0.00146884],
       [0.01333323],
       [0.00039895],
       [0.02183425],
       [0.15113945],
       [0.00064366],
       [0.00659913],
       [0.01056644],
       [0.050

In [18]:
# Y for treated (X = 1)
treated_y_mean = treated['Y'].mean()

# Y for matched controls (matched_control_Y was created earlier)
matched_control_y_mean = treated['matched_control_Y'].mean()

# Estimated treatment effect
effect = treated_y_mean - matched_control_y_mean
print(f"Estimated effect: {effect:.4f}")


Estimated effect: 0.5434


In [4]:
df = pd.read_csv('homework_2.1.csv')


In [5]:
df.head()

,Unnamed: 0,time,G1,G2,G3
0,0,0,0.882026,1.441575,0.065409
1,1,1,0.210079,-0.163880,0.140310
2,2,2,0.509369,-0.115242,0.819830
3,3,3,1.150447,1.014698,0.607632
4,4,4,0.973779,-0.046562,0.610066


In [6]:
import pandas as pd
import statsmodels.api as sm

# Assume df is your DataFrame with columns x1, x2, x3, and Y
X = df[['G1', 'G2', 'G3']]
X = sm.add_constant(X)  # adds the intercept term
y = df['time']

# Fit the model
model = sm.OLS(y, X).fit()

# View results with t-statistics
print(model.summary())

# If you just want t-statistics:
print(model.tvalues)

                            OLS Regression Results                            
Dep. Variable:                   time   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     24.68
Date:                Mon, 02 Jun 2025   Prob (F-statistic):           6.38e-12
Time:                        02:53:02   Log-Likelihood:                -449.58
No. Observations:                 100   AIC:                             907.2
Df Residuals:                      96   BIC:                             917.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.4504      5.057      3.253      0.0